In [17]:
import multiprocessing as mp
import psutil 
from rosette import Rosette
import pyvista as pv
import concurrent.futures
from tqdm import tqdm
from functools import partial
from itertools import repeat
# from multiprocessing import Pool, freeze_support
from multiprocess import Pool, freeze_support

In [10]:
mp.cpu_count()

10

In [11]:
n_core_physical = psutil.cpu_count(logical = False)
n_core_logical = psutil.cpu_count(logical = True)
print("# physical cores = ", n_core_physical)
print("# logical cores = ", n_core_logical)

# physical cores =  10
# logical cores =  10


In [12]:
%%timeit
# create a single rosette

# set geoemetric parameters
a = 0.5 # half max length across basal face
c =  2.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 8 # number of bullet arms
test = Rosette(a, c, r0, h0, hp, n_arms)

11 ms ± 402 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
%%time
# create rosette 100 times 
# set geoemetric parameters
a = 0.5 # half max length across basal face
c =  2.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 12 # number of bullet arms
N = 1_000
for i in tqdm(range(N)):
    test = Rosette(a, c, r0, h0, hp, n_arms)

100%|██████████| 1000/1000 [00:13<00:00, 73.59it/s]

CPU times: user 13.1 s, sys: 970 ms, total: 14.1 s
Wall time: 13.6 s


In [28]:
%%time
# create rosette 100 times, in parallel
n_cores = psutil.cpu_count(logical = False)
# set geoemetric parameters
a = 0.5 # half max length across basal face
c =  2.0 # half max length across prism face
r0 = 1.0 # radius of center sphere
h0 = 0.25 # penetration depth of bullets
hp = 0.75 # heights of pyramid of bullets
n_arms = 12 # number of bullet arms
N = 1_000
def create_rosette(N):
    test = Rosette(a, c, r0, h0, hp, n_arms)

def main():
    with Pool(n_cores) as pool:
        pool.map(create_rosette, range(N))

if __name__=="__main__":
    main()


CPU times: user 18.9 ms, sys: 35 ms, total: 53.9 ms
Wall time: 2.04 s


In [8]:
from functools import partial
from itertools import repeat
# from multiprocessing import Pool, freeze_support
from multiprocess import Pool, freeze_support

def func(a, b):
    return a + b

def main():
    a_args = [1,2,3]
    second_arg = 1
    with Pool() as pool:
        L = pool.starmap(func, [(1, 1), (2, 1), (3, 1)])
        M = pool.starmap(func, zip(a_args, repeat(second_arg)))
        N = pool.map(partial(func, b=second_arg), a_args)
        assert L == M == N

if __name__=="__main__":
    freeze_support()
    main()